In [106]:
import requests


def parse_name(x):
  url = "http://localhost:30888/lab/entities/parse"
  body = {
    "user_input": x,
    "entities": "Nombre::SegundoNombre::Apellido"
  }
  
  response = requests.post(url, json=body)

  
  if response.status_code == 200:
    parsed_response = response.json()
    print(parsed_response)
    try:
      entities = [parsed_response['Nombre'], parsed_response['SegundoNombre'], parsed_response['Apellido']]
      return entities

    except KeyError:
      print("No se pudo extraer bien el nombre")
      entities = [x,"", ""]
      return entities



In [108]:
n,sn, a = parse_name("AACCMA")
a

{'Nombre': 'A', 'SegundoNombre': ''}
No se pudo extraer el nombre


''

In [64]:
import requests


def create_invite(name):
    url = "http://localhost:30888/lab/systemusergpt4"
    payload = {
        "system": "creá un recordatorio personalizado para el usuario a la saiaconf tiene que ser corta y con algunos emojis. Avisa que hay que llegar antes de las 14 hs para reservar lugar. Se breve. La SAIAConf se realizará el sábado 25 de noviembre en la sede de la UTN en Av. Medrano 951. Recordale al usuario que después hay after. Habla en castellano argentino.",
        "user": name,
    }
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        print("Invitation created successfully")
        return response.json()
    else:
        print("Failed to create invitation")

In [65]:
invite = create_invite("Juan Perez")
invite['content']

Invitation created successfully


'¡Ey, Juan! 👋 No te olvides que este sábado 25/11 es la SAIAConf en la UTN de Medrano 951. 🚀 Llegá antes de las 14 hs para asegurar tu lugar. 🕑 Acordate que después tenemos el after. ¡No te lo pierdas! 🎉 Nos vemos allá, ¡abrazo!'

In [43]:
def clean_number(x):
    url = "http://localhost:30888/lab/entities/parse"
    body = {"user_input": x, "entities": "NumeroCelular"}

    response = requests.post(url, json=body)
    if response.status_code == 200:
        return response.json()["NumeroCelular"]
    else:
        return None

In [44]:
c = clean_number("mi numero es 301 123 4567")
c

'+5493011234567'

In [60]:
first_name, second_name, family_name = parse_name("juan pablo gonzalez")
family_name

{'Nombre': 'Juan Pablo', 'SegundoNombre': '', 'Apellido': 'Gonzalez'}


'Gonzalez'

In [2]:
import pandas as pd
import requests
import re

# Define the functions clean_number(), parse_name(), and create_invite() here

# Read the CSV file
df = pd.read_csv('../data/loaded.csv')




In [3]:
df['phone_number'] = df['phone_number'].str.replace('p:', '')


In [111]:
df["phone_number"].head(5)

0     +56965909598
1    +541168481103
2    +541161606434
3    +543754462815
4    +542944247890
Name: phone_number, dtype: object

In [112]:
# Iterate over each row
for index, row in df.iterrows():
    #print the row number
    print("Row Number: ", index)
    # Clean the phone number
    cleaned_number = clean_number(row["phone_number"])

    # Parse the full name
    first_name, second_name, family_name = parse_name(row["full_name"])

    # Create the invite
    # invite = create_invite(first_name)
    # print(invite)

    # Prepare the data to be posted
    data = {
        "first_name": first_name,
        "second_name": second_name,
        "family_name": family_name,
        "email": row["email"],
        "whatsapp": cleaned_number,
    }

    # Post the data to the endpoint
    response = requests.post("https://cms.saia.ar/items/leads", json=data)

    # Check the response status
    if response.status_code == 200:
        print(response.json()['data']['id'])
        print("Data posted successfully")

    else:
        print("Failed to post data")

Row Number:  0
{'Nombre': 'Patricio', 'SegundoNombre': '', 'Apellido': 'Soto'}
Failed to post data
Row Number:  1
{'Nombre': 'Rafael', 'SegundoNombre': '', 'Apellido': 'Costa'}
Failed to post data
Row Number:  2
{'Nombre': 'Virginias', 'SegundoNombre': '', 'Apellido': 'Carbone'}
Failed to post data
Row Number:  3
{'Nombre': 'Aldana', 'SegundoNombre': 'Mariel', 'Apellido': 'Rolin'}
Failed to post data
Row Number:  4
{'Nombre': 'Rubi', 'SegundoNombre': '', 'Apellido': 'Schram'}
Failed to post data
Row Number:  5
{'Nombre': 'Guillermo', 'SegundoNombre': 'Oscar', 'Apellido': ''}
Failed to post data
Row Number:  6
{'Nombre': 'Andres', 'SegundoNombre': '', 'Apellido': 'Millan'}
Failed to post data
Row Number:  7
{'Nombre': 'Tomás', 'SegundoNombre': '', 'Apellido': 'Villar'}
Failed to post data
Row Number:  8
{'Nombre': 'Ezequiel', 'SegundoNombre': '', 'Apellido': ''}
Failed to post data
Row Number:  9
{'Nombre': 'Julian', 'SegundoNombre': '', 'Apellido': 'Baigorria'}
Failed to post data
Row 

In [4]:
import requests

for index, row in df.iterrows():
    email = row["email"]
    url = f'https://cms.saia.ar/items/leads?filter={{"email":{{"_eq":"{email}"}}}}'

    # Get the ID
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data and "data" in data and len(data["data"]) > 0:
            contact_id = data["data"][0]["id"]
        else:
            print(f"No contact found for email: {email}")
            continue
    else:
        print(f"Failed to get contact for email: {email}")
        continue

    # Patch the contact
    patch_url = f"https://cms.saia.ar/items/leads/{contact_id}"
    knows_saia_from = row["campaign_name"]
    

    body = {
        "saiaconf_ad": True,
        "knows_saia_from": knows_saia_from,
    }

    response = requests.patch(patch_url, json=body)
    if response.status_code == 200:
        print(f"Contact patched successfully for email: {email}")
    else:
        print(f"Failed to patch contact for email: {email}")

Contact patched successfully for email: patricio@duoapp.cl
Contact patched successfully for email: rafaelcosta19@hotmail.com
Contact patched successfully for email: carbonevirginia@gmail.com
Contact patched successfully for email: aldanarolingm@gmail.com
Contact patched successfully for email: rubi.schram@gmail.com
Contact patched successfully for email: guillermooscar1956@gmail.com
Contact patched successfully for email: construccionescamsrl@gmail.com
Contact patched successfully for email: tomasvillar88@gmail.com
Contact patched successfully for email: ezeoller@gmail.com
Contact patched successfully for email: baigorriajulian@gmail.com
Contact patched successfully for email: daryuscarambula@gmail.com
Contact patched successfully for email: nahirsoneira7@gmail.com
Contact patched successfully for email: nectarsoy222@gmail.com
Contact patched successfully for email: ma.inespalacios@gmail.com
Contact patched successfully for email: alexdago@gmail.com
Contact patched successfully for ema